
2.Napisać skrypt w języku Python do wczytywania kolejnych obrazów oraz wycinania z nich
fragmentów (czyli próbek tekstury) o zadanym rozmiarze (np. 128 x 128). Rozmiar
wczytywanych obrazów powinien być parametrem napisanej funkcji. Wycięte fragmenty zapisać
do odpowiednich katalogów, tzn. każdą grupę fragmentów umieścić w osobnym folderze
reprezentującym daną kategorię tekstury.

In [ ]:
from PIL import Image, ImageFilter
import os

def cut_texture(img_folder, save_folder, img_size=(512, 512), sample_size=(128, 128)):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)


    for filename in os.listdir(img_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(img_folder, filename)
            img = Image.open(img_path)

          
            img = img.resize(img_size, Image.Resampling.LANCZOS)

           
            xnum = img.size[0] // sample_size[0]
            ynum = img.size[1] // sample_size[1]

            for i in range(xnum):
                for j in range(ynum):
                  
                    box = (i * sample_size[0], j * sample_size[1], (i + 1) * sample_size[0], (j + 1) * sample_size[1])
                    sample = img.crop(box)
            
                    sample_filename = f"{filename[:-4]}_sample_{i}_{j}.jpg"
                    sample_path = os.path.join(save_folder, sample_filename)
                    sample.save(sample_path)




img_folder = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3"
save_folder = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3_samples"
cut_texture(img_folder, save_folder, img_size=(512, 512), sample_size=(128, 128))

3.Napisać skrypt do wczytywania próbek tekstury i wyznaczania dla nich cech tekstury na
podstawie modelu macierzy zdarzeń (ang. grey-level co-occurrence matrix) za pomocą
odpowiednich funkcji z pakietu scikit-image. Przed obliczeniem cech, każdy obraz przekształcić
do skali szarości oraz zmniejszyć głębię jasności do 5 bitów (64 poziomy). Wyznaczyć
następujące cechy:
a. dissimilarity, correlation, contrast, energy, homogeneity, ASM.
b. Przyjąć 3 odległości pikseli: 1, 3, 5
c. oraz 4 kierunki: 0, 45, 90 I 135 stopni (zakładamy symetrię kierunków).
d. Każdy wektor cech uzupełnić o nazwę kategorii tekstury
4. Zapisać zbiór wektorów danych do pliku csv. Można wykorzystać pakiet Pandas.

In [3]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, util
from skimage.feature import greycomatrix, greycoprops

def extract_texture_features(img_folders, csv_path):
    distances = [1, 3, 5]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0, 45, 90, 135 stopni
    properties = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']

    
    rows = []

  
    for img_folder in img_folders:
        for filename in os.listdir(img_folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(img_folder, filename)
                img = io.imread(img_path, as_gray=True)
    
                img = util.img_as_ubyte(img)
                img = img // 4

                glcm = greycomatrix(img, distances, angles, levels=64, symmetric=True, normed=True)
                feature_values = [greycoprops(glcm, prop).ravel().mean() for prop in properties]

                row = [filename] + feature_values
                rows.append(row)

    df = pd.DataFrame(rows, columns=['filename'] + properties)

    df.to_csv(csv_path, index=False)


img_folders = [
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO1_samples",
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO2_samples",
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3_samples"
]
csv_path = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\texture_features.csv"
extract_texture_features(img_folders, csv_path)

5. Napisać skrypt do klasyfikacji wektorów cech z wykorzystaniem dowolnego algorytmu
klasyfikacji danych dostępnego w pakiecie scikit-learn (np. support vector machines, K nearest
neighbors). Uczenie przeprowadzić dla wyodrębnionego zbioru treningowego, a testowanie dla
zbioru testowego. Obliczyć i wyświetlić na ekranie wyznaczoną dokładność klasyfikatora.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv(r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\texture_features.csv")



# data['class'] = data['filename'].apply(lambda x: x.split('_')[2])
data['class'] = 'unknown'

X = data[['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']]


y = data['class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność klasyfikatora: {accuracy*100:.2f}%")
